HW6.2

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
%matplotlib inline
import torch
import torch.optim as optim
import time
from sklearn.datasets import load_breast_cancer

In [19]:
cancer_data = load_breast_cancer()
sample = cancer_data.data
sample.shape

(569, 30)

In [20]:
sample_df = pd.DataFrame(sample)
sample_df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.8,1001.0,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189


In [21]:
cancer_labels = cancer_data.target

In [22]:
labels = np.reshape(cancer_labels, (569, 1))
final_data = np.concatenate([sample, labels], axis = 1)
final_data.shape

(569, 31)

In [23]:
cancer_dataset = pd.DataFrame(final_data)
features = cancer_data.feature_names
features

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [24]:
features_labels = np.append(features, 'label')
cancer_dataset.columns = features_labels
cancer_dataset.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,label
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0.0
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0.0
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0.0


In [25]:
X = cancer_dataset.iloc[:, 0:30].values
Y = cancer_dataset.iloc[:, 30].values

In [28]:
t_x = torch.tensor(X).unsqueeze(1) # u=x
t_y = torch.tensor(Y).unsqueeze(1) # c=y
t_x.shape

torch.Size([569, 1, 30])

In [29]:
n_samples = t_y.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [30]:
t_x_train = t_x[train_indices].to(torch.float32)
t_y_train = t_y[train_indices].to(torch.float32)

t_x_val = t_x[val_indices].to(torch.float32)
t_y_val = t_y[val_indices].to(torch.float32)

In [31]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_x_train, t_x_val,
                  t_y_train, t_y_val):
    for epoch in range(1, n_epochs + 1):
        epoch_start_time = time.time()
        t_p_train = model(t_x_train) # <1>
        loss_train = loss_fn(t_p_train, t_y_train)

        t_p_val = model(t_x_val) # <1>
        loss_val = loss_fn(t_p_val, t_y_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()
        epoch_end_time = time.time()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}"
            f" Training time: {(epoch_end_time - epoch_start_time):.4f} seconds")

In [33]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [38]:
linear_model = nn.Linear(11, 1) # <1>

seq_model = nn.Sequential(
            nn.Linear(30, 32), # <1>
            nn.Tanh(),
            nn.Linear(32, 2), # <2>
            nn.LogSoftmax(dim=1)) 

optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = loss_fn,
    t_x_train = t_x_train,
    t_x_val = t_x_val, 
    t_y_train = t_y_train,
    t_y_val = t_y_val)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 0.6250, Validation loss 0.6372 Training time: 0.0077 seconds
Epoch 1000, Training loss 0.6250, Validation loss 0.6372 Training time: 0.0035 seconds
Epoch 2000, Training loss 0.6250, Validation loss 0.6372 Training time: 0.0041 seconds
Epoch 3000, Training loss 0.6250, Validation loss 0.6372 Training time: 0.0039 seconds
Epoch 4000, Training loss 0.6250, Validation loss 0.6372 Training time: 0.0047 seconds
Epoch 5000, Training loss 0.6250, Validation loss 0.6372 Training time: 0.0036 seconds

Parameter containing:
tensor([[-0.0682, -0.1942, -0.1495,  0.2004,  0.1315,  0.0435, -0.0051,  0.1686,
         -0.2365, -0.2529, -0.0297]], requires_grad=True)
Parameter containing:
tensor([0.0659], requires_grad=True)


In [35]:
seq_model = nn.Sequential(
            nn.Linear(30, 32), # <1>
            nn.Tanh(),
            nn.Linear(32, 16), # <2>
            nn.Tanh(),
            nn.Linear(16, 32), # <3>
            nn.Tanh(),
            nn.Linear(32, 2),
            nn.LogSoftmax(dim=1))

optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = loss_fn,
    t_x_train = t_x_train,
    t_x_val = t_x_val, 
    t_y_train = t_y_train,
    t_y_val = t_y_val)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 0.6250, Validation loss 0.6372 Training time: 0.0211 seconds
Epoch 1000, Training loss 0.6250, Validation loss 0.6372 Training time: 0.0049 seconds
Epoch 2000, Training loss 0.6250, Validation loss 0.6372 Training time: 0.0050 seconds
Epoch 3000, Training loss 0.6250, Validation loss 0.6372 Training time: 0.0040 seconds
Epoch 4000, Training loss 0.6250, Validation loss 0.6372 Training time: 0.0045 seconds
Epoch 5000, Training loss 0.6250, Validation loss 0.6372 Training time: 0.0050 seconds

Parameter containing:
tensor([[ 0.2639,  0.2689,  0.2464,  0.1706, -0.1389,  0.1034, -0.2330,  0.1488,
         -0.0166, -0.0615,  0.0363]], requires_grad=True)
Parameter containing:
tensor([-0.0595], requires_grad=True)
